In [1]:
import pandas as pd
import numpy as np
import cupy as cp
from tqdm import tqdm

In [2]:
!nvidia-smi

Tue Mar  1 07:30:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 5000     On   | 00000000:01:00.0  On |                  N/A |
| N/A   48C    P8    11W /  N/A |  15200MiB / 16116MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   39 bits physical, 48 bits virtual
CPU(s):                          16
On-line CPU(s) list:             0-15
Thread(s) per core:              2
Core(s) per socket:              8
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           158
Model name:                      Intel(R) Xeon(R) E-2286M  CPU @ 2.40GHz
Stepping:                        13
CPU MHz:                         2400.000
CPU max MHz:                     5000.0000
CPU min MHz:                     800.0000
BogoMIPS:                        4800.00
Virtualization:                  VT-x
L1d cache:                       256 KiB
L1i cache:                       256 KiB
L2 cache:                        2 MiB
L3 cache:                     

In [4]:
!cat /proc/meminfo

MemTotal:       65679168 kB
MemFree:        17731956 kB
MemAvailable:   49960088 kB
Buffers:         1307632 kB
Cached:         28791124 kB
SwapCached:          116 kB
Active:          8395076 kB
Inactive:       35082748 kB
Active(anon):      45364 kB
Inactive(anon): 13786812 kB
Active(file):    8349712 kB
Inactive(file): 21295936 kB
Unevictable:          56 kB
Mlocked:              48 kB
SwapTotal:       2097148 kB
SwapFree:        2094588 kB
Dirty:              2952 kB
Writeback:             0 kB
AnonPages:      13379548 kB
Mapped:          1581356 kB
Shmem:            463568 kB
KReclaimable:    3314588 kB
Slab:            3950452 kB
SReclaimable:    3314588 kB
SUnreclaim:       635864 kB
KernelStack:       28240 kB
PageTables:       101424 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    34936732 kB
Committed_AS:   35458372 kB
VmallocTotal:   34359738367 kB
VmallocUsed:      117784 kB
VmallocChunk:          0 kB
Percpu:          

In [5]:
from scipy.fft import fft2, rfft2, hfft2
#from cupyx.scipy.fft import fft2, rfft2, hfft2

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/train.csv.gz
../input/sample_submission.csv
../input/test.csv.gz


In [6]:
train = pd.read_csv('../input/train.csv.gz')
test = pd.read_csv('../input/test.csv.gz')
col = ['pixel%d'%i for i in range(784)]
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X_train = train[col].values
X_test = test[col].values

In [9]:
X_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0, 188, 255,  94,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0, 191, 250, 253,  93,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [10]:
X_train.shape

(42000, 784)

In [11]:
%%time
X_train_rffts = []

for i in range(X_train.shape[0]):
    x0 = X_train[i].reshape((28,28))
    x0_rfft = np.fft.rfft2(x0)
    X_train_rffts.append(x0_rfft.flatten())

CPU times: user 1 s, sys: 5.83 ms, total: 1.01 s
Wall time: 1.01 s


In [12]:
X_train_rffts = np.array(X_train_rffts)

In [13]:
X_train_rffts.shape

(42000, 420)

In [15]:
np.real(X_train_rffts)

array([[ 1.66490000e+04, -1.08257306e+04,  6.76963766e+02, ...,
        -5.16570816e+01,  9.95065039e+01,  1.88794665e+02],
       [ 4.46090000e+04, -1.98178794e+04, -1.18997008e+04, ...,
         1.33841297e+03, -7.13075176e+02, -6.97117791e+02],
       [ 1.34250000e+04, -1.31473875e+04,  1.23414492e+04, ...,
         2.20034879e+02, -7.24335318e+02,  1.15771960e+03],
       ...,
       [ 3.15040000e+04, -1.93155379e+04,  1.26495096e+03, ...,
         1.37020337e+02,  2.34737792e+01, -5.48593545e+02],
       [ 2.63810000e+04, -1.80486774e+04,  3.58099480e+03, ...,
         1.46007899e+02,  7.71211686e+01, -1.34041127e+02],
       [ 1.81780000e+04, -1.37086207e+04,  4.32974446e+03, ...,
         4.61723260e+02, -1.35187041e+02, -3.31860635e+02]])

In [16]:
np.imag(X_train_rffts)

array([[    0.        ,  -393.92361504,  1416.36972316, ...,
          241.52381929,   181.97248153,  -354.84484368],
       [    0.        , -2173.50795916,  -951.60562691, ...,
          700.01269407,  -436.61944812,   634.70094532],
       [    0.        ,  -758.03364097,  1421.19349145, ...,
           46.86767769,   163.31789416,  -271.83755267],
       ...,
       [    0.        ,  3266.23522981, -5801.32930255, ...,
           -6.11915063,  -281.26660143,    13.54125169],
       [    0.        ,   349.01223171,   708.01409315, ...,
         -603.72295784,   312.04267895,  -121.72458798],
       [    0.        ,  1737.79385063, -4326.32789543, ...,
           17.4690778 ,   163.55507931, -1032.33154867]])

In [17]:
X_train_rffts = np.hstack([np.real(X_train_rffts), np.imag(X_train_rffts)])

In [18]:
X_train_rffts.shape

(42000, 840)

In [19]:
%%time
X_test_rffts = []

for i in range(X_test.shape[0]):
    x0 = X_test[i].reshape((28,28))
    x0_rfft = np.fft.rfft2(x0)
    X_test_rffts.append(x0_rfft.flatten())
    
X_test_rffts = np.array(X_test_rffts)
X_test_rffts = np.hstack([np.real(X_test_rffts), np.imag(X_test_rffts)])
X_test_rffts.shape

CPU times: user 801 ms, sys: 52.2 ms, total: 853 ms
Wall time: 858 ms


(28000, 840)

In [22]:
train_rffts = pd.DataFrame(data = X_train_rffts, columns = ['column%d'%i for i in range(840)])
train_rffts['label'] = train['label']
train_rffts.head()

,column0,column1,column2,column3,column4,column5,column6,column7,column8,column9,...,column831,column832,column833,column834,column835,column836,column837,column838,column839,label
0,16649.0,-10825.730588,676.963766,2972.747410,-331.791733,-1630.186080,800.741748,80.0,145.728115,-363.814434,...,-761.691003,-203.194784,423.136653,153.183586,-180.555275,-254.919830,241.523819,181.972482,-354.844844,1
1,44609.0,-19817.879430,-11899.700821,7152.010894,6192.999503,-2268.783599,-2748.228295,488.0,390.744388,46.205112,...,235.124052,450.535508,303.072135,-393.539691,-723.869427,101.385628,700.012694,-436.619448,634.700945,0
2,13425.0,-13147.387496,12341.449201,-11083.963364,9490.534722,-7699.389592,5852.892616,-4080.0,2482.590550,-1127.786479,...,662.742084,-845.319580,886.655304,-789.146911,581.917730,-314.556589,46.867678,163.317894,-271.837553,1
3,15025.0,-6462.372814,-6805.507220,7533.698456,946.579987,-4640.311023,1487.771136,747.0,195.898912,-430.387364,...,-223.685066,-2221.829438,1950.690155,871.882828,-2187.637017,734.000824,905.695550,-980.830446,193.303266,4
4,51093.0,-18393.458365,-16356.968576,3604.196760,6944.535309,678.005820,-527.521874,-492.0,-1811.020731,-1175.359535,...,184.710843,569.723654,-233.257547,-474.614050,-80.456407,106.090312,-161.727479,587.194889,463.961436,0


In [23]:
test_rffts = pd.DataFrame(data = X_test_rffts, columns = ['column%d'%i for i in range(840)])
test_rffts.head()

,column0,column1,column2,column3,column4,column5,column6,column7,column8,column9,...,column830,column831,column832,column833,column834,column835,column836,column837,column838,column839
0,39404.0,-18906.732622,-5146.300787,2577.674812,2457.863642,-983.171676,430.310252,580.0,-194.647951,-292.851020,...,-1299.981139,317.293434,679.820599,922.390899,-278.980452,-156.012490,413.671291,21.873111,-529.194641,-839.691826
1,39933.0,-14062.587657,-10110.726687,-1944.781456,4802.327853,3623.311516,553.934803,-2686.0,-1332.710204,440.961800,...,-615.680852,125.183512,-289.338874,-513.402691,441.501298,-303.954786,-392.177241,583.633694,459.358952,179.508930
2,14870.0,-10600.551679,2488.382337,1986.643869,-1566.992468,-149.196973,782.693282,-442.0,-81.304631,236.795895,...,-484.367896,1142.244304,-213.121293,-548.827656,315.846561,-80.248822,170.703029,49.303205,-504.126694,545.387835
3,16208.0,-10432.603182,-355.082185,5516.014909,-2345.370932,-2039.217306,1922.463636,928.0,-1725.810637,-44.289352,...,759.858315,-101.809821,-166.110449,145.195583,-14.403689,113.296886,-150.058066,17.274584,48.704123,47.168918
4,24294.0,-19077.881478,7515.914804,2306.642814,-5745.772765,4066.635736,-996.487721,-1071.0,1690.605696,-1276.379564,...,1810.856929,-1946.708622,651.723671,236.083503,-298.731667,146.513436,-115.769298,48.724877,145.017165,-282.299158


In [24]:
train_rffts.to_csv('../input/train_rffts.csv.gz', compression='gzip')
test_rffts.to_csv('../input/test_rffts.csv.gz', compression='gzip')